In [178]:
import pandas as pd
from ftplib import FTP
from datetime import datetime
import time
import numpy as np
import pymongo
from pymongo import MongoClient

from geopy import distance

import get_recent_days as gtdys
import muni_etl
import labelling as lblng

In [179]:
client = MongoClient('localhost', 27017)

db = client['testavl']
in_coll = db['clean']
out_coll = db['clean_out']

In [180]:
out_coll.delete_many({})
out_coll.count()

0

In [181]:
labeler = lblng.Labeling(in_coll, out_coll)

In [182]:
labeler.label_single_starts()

Start Count:  45


Duplicate ID Count:  0




In [183]:
labeler.label_trips()

Total Good Trips:  40


Total Emtpy Trips:  0


Total Sparse Trips:  3


Total Dense Trips:  1


Total 'Endless' Trips:  2


In [184]:
out_coll.count()

3030

In [132]:
len(out_coll.find().distinct('trip_id_iso'))

45

In [144]:
params = [
            {"$group" : { "_id": "$trip_id_iso", "count": { "$sum": 1 }}},
            {"$match": {"count" : {"$gt": 1}}}
         ]
hoop = out_coll.aggregate(params)

for item in hoop:
    print (item)

{'_id': '7253809_2016-11-19', 'count': 2}


In [146]:
for pica in out_coll.find({'trip_id_iso': '7253809_2016-11-19'}):
    print (pica)

{'_id': ObjectId('5ae20f963ad39e1a4fce3782'), 'HEADING': '79.0', 'LATITUDE': '37.78693', 'LONGITUDE': '-122.45655', 'PREDICTABLE': '1', 'REPORT_TIME': '11/19/2016 22:00:55', 'REV': '1526', 'SPEED': '0.833', 'TRAIN_ASSIGNMENT': '3304', 'VEHICLE_TAG': '5445', 'sched_time_diff_seconds': 55, 'service_id': 2, 'time_stamp': 1479621655.0, 'trip_id': 7253809, 'trip_id_iso': '7253809_2016-11-19', 'trip_start': 1}
{'_id': ObjectId('5ae20f963ad39e1a4fce381f'), 'HEADING': '82.0', 'LATITUDE': '37.7869', 'LONGITUDE': '-122.4565', 'PREDICTABLE': '1', 'REPORT_TIME': '11/19/2016 23:59:28', 'REV': '1526', 'SPEED': '0.0', 'TRAIN_ASSIGNMENT': '3304', 'VEHICLE_TAG': '5445', 'sched_time_diff_seconds': 7168, 'service_id': 2, 'time_stamp': 1479628768.0, 'trip_id': 7253809, 'trip_id_iso': '7253809_2016-11-19', 'trip_start': 1}


In [147]:
7168/60

119.46666666666667

In [148]:
30 * 60

1800

In [102]:
trips = labeler.trip_blocks[labeler.trip_blocks['service_id'] == 2]
trips.shape

(58, 7)

In [37]:
labeler.sched_trps.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
476240,7253651,24:00:00,24:00:00,6293,1,,,,
476241,7253651,24:00:56,24:00:56,3879,2,,,,
476242,7253651,24:01:32,24:01:32,3852,3,,,,
476243,7253651,24:02:28,24:02:28,3644,4,,,,
476244,7253651,24:03:39,24:03:39,3645,5,,,,


In [59]:
lst_seq = labeler.sched_trps.groupby('trip_id')['stop_sequence'].max().unique()

In [61]:
last_stop_id = labeler.sched_trps[labeler.sched_trps['stop_sequence'].isin(lst_seq)]['stop_id'].unique()
last_stop_id

array([3282, 3511])

In [62]:
labeler.stop_sched[labeler.stop_sched['stop_id'].isin(last_stop_id)]

,stop_id,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url
353,3282,16th St & Bryant St,,37.765595,-122.410408,,
574,3511,25th St & Potrero Ave,,37.751407,-122.406680,,


In [151]:
import folium

map_2 = folium.Map(location=[37.770373, -122.436064],
                   tiles='Stamen Terrain',
                   zoom_start=13)

#16 Bryant
last_stop = (37.765595, -122.410408)

# #25 Potrero
# last_stop = (37.751407, -122.406680)

for data in labeler.testdocs['7253850_2016-11-19']:

    lat = data['LATITUDE']
    lon = data['LONGITUDE']

    rprt_time = data['REPORT_TIME']
    
    dist = distance.distance(last_stop, (float(lat), float(lon))).m

    folium.Marker([float(lat), float(lon)], popup=rprt_time + "  " + str(dist)).add_to(map_2)



# folium.Marker([endstp_latlon[0], endstp_latlon[1]], icon=folium.Icon(color='red')).add_to(map_2)
    
map_2
